In [1]:
from nutils import*
from nutils.pointsseq import PointsSequence
import numpy as np
import os
import vtk
from vtk.util import numpy_support

In [ ]:
def GetLumenMesh(fname):
    reader = vtk.vtkXMLUnstructuredGridReader()
    reader.SetFileName(fname)
    reader.Update()
    return reader.GetOutput()

def GetMeshPoints(mesh):
    pts = mesh.GetPoints().GetData()
    pts_array = numpy_support.vtk_to_numpy(pts)
    return pts_array

def GetMeshCells(mesh):
    cells = mesh.GetCells().GetData()
    cells_array = numpy_support.vtk_to_numpy(cells)
    return cells_array
    
def GetMeshData(mesh, data_name):
    data = mesh.GetPointData().GetArray(data_name)
    data_array = numpy_support.vtk_to_numpy(data)
    return data_array

In [2]:
class Triangle:
    def __init__(self, v0, v1, v2):
        self.v0 = v0
        self.v1 = v1
        self.v2 = v2

class Vec3:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def sub(self, v):
        return Vec3(self.x - v.x,
                    self.y - v.y,
                    self.z - v.z)

    def dot(self, v):
        return self.x * v.x + self.y * v.y + self.z * v.z

    def cross(self, v):
        return Vec3(self.y * v.z - self.z * v.y,
                    self.z * v.x - self.x * v.z,
                    self.x * v.y - self.y * v.x)

    def length(self):
        return math.sqrt(self.x * self.x +
                         self.y * self.y +
                         self.z * self.z)

    def normalize(self):
        l = self.length()
        return Vec3(self.x / l, self.y / l, self.z / l)


class Ray:
    def __init__(self, orig=None, direction=None):
        self.orig = orig
        self.direction = direction


def ray_triangle_intersect(r, tri tol=1e-6):
    v0v1 = tri.v1.sub(tri.v0)
    v0v2 = tri.v2.sub(tri.v0)
    pvec = r.direction.cross(v0v2)

    det = v0v1.dot(pvec)

    if det < tol:
        return False

    invDet = 1.0 / det
    tvec = r.orig.sub(tri.v0)
    u = tvec.dot(pvec) * invDet

    if u < 0 or u > 1:
        return False

    qvec = tvec.cross(v0v1)
    v = r.direction.dot(qvec) * invDet

    if v < 0 or u + v > 1:
        return False

    return True # v0v2.dot(qvec) * invDet

In [3]:
class PolygonInteriorIndicator(function.Pointwise):
    @types.apply_annotations
    def __init__(self, mesh, *args:function.asarrays):
        self.imspacing = image_data.spacing
        self.imorigin = image_data.origin
        self.imvals = image_data.vals
        self.imshape = image_data.shape
        retval = self.evalf(*[np.ones((), dtype=arg.dtype) for arg in args])
        shapes = set(arg.shape for arg in args)
        assert len(shapes) == 1, 'pointwise arguments have inconsistent shapes'
        shape, = shapes
        self.args = args
        super().__init__(args=args, shape=shape, dtype=retval.dtype)
        
    def evalf(x,y, val_in, val_out):
        shape = x.shape
        if shape == ():
            return x
        else:
            vals = np.zeros(shape)
            for j in range(shape[0]):
                pt = [x[j] , y[j]]
                nIntersections = 0
                p1 = sorted_pts[0]
                for i in range(1,len(sorted_pts)+1):
                    p2 = sorted_pts[i % len(sorted_pts)]
                    if pt[1] > min([p1[1],p2[1]]):
                        if pt[1] <= max([p1[1],p2[1]]):
                            if pt[0] <= max([p1[0],p2[0]]):
                                if p1[1] != p2[1]:
                                    xint = (pt[1] - p1[1]) * (p2[0] - p1[0]) / (p2[1] - p1[1]) + p1[0]
                                    if p1[0] == p2[0] or pt[0] <= xint:
                                        nIntersections+=1
                    p1 = p2
                if nIntersections % 2 == 0:
                    vals[j] = val_out
                else:
                    vals[j] = val_in
            return vals 
                    
    def _derivative(self, var, seen):
        return np.zeros(self.shape + var.shape)


SyntaxError: invalid syntax (<ipython-input-3-a75c78e8bc50>, line 3)

In [ ]:
class VoxelImageFunction(function.Array):
    @types.apply_annotations
    def __init__(self, image_data, *args:function.asarrays):
        self.imspacing = image_data.spacing
        self.imorigin = image_data.origin
        self.imvals = image_data.vals
        self.imshape = image_data.shape
        retval = self.evalf(*[np.ones((), dtype=arg.dtype) for arg in args])
        shapes = set(arg.shape for arg in args)
        assert len(shapes) == 1, 'pointwise arguments have inconsistent shapes'
        shape, = shapes
        self.args = args
        super().__init__(args=args, shape=shape, dtype=retval.dtype)

    def PointToVoxelCoordinates(self, pt):
        x = np.floor( (pt + self.imspacing / 2 - self.imorigin) / self.imspacing )
        return x.astype(int)

    def VoxelCoordinatesToID(self, coords):
        return coords[0] + coords[1] * self.imshape[0] + coords[2] * self.imshape[0] * self.imshape[1]

    def evalf(self, x, y, z):
        shape = x.shape
        if shape == ():
            return x
        else:
            vals = np.zeros(shape)
            for i in range(shape[0]):
                pt = np.array([x[i], y[i], z[i]])
                voxel_coords = self.PointToVoxelCoordinates(pt)
                voxel_ID = self.VoxelCoordinatesToID(voxel_coords)
                vals[i] = 1 #self.imvals[voxel_ID]
            return vals

    def _derivative(self, var, seen):
        return np.zeros(self.shape+var.shape)